# Availability Groups

***You will need to have followed the steps in the 00-CreateContainers notebook to use this notebook***

We have written a book which will give you a brilliant introduction to dbatools. It's called dbatools in a Month of Lunches and you can find it at https://beard.media/book

Setting up availabilty groups is hard

with dbatools it is a little easier :-)

The next block sets the variables for the instances and folder paths for this Notebook and checks the connection - Refer to the first notebook for any issues

In [ ]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'
$SqlInstances = 'localhost,15592', 'localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Write-Output " Creating connection to the containers"
try {
    $SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
    $SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
    Write-Output "We have a connection to the containers"

}
catch {
    Write-Output "You haven't got a connection to the containers - Either they are still upgrading in which case try again in 30 seconds or the containers have not come up correctly"
    Write-Output "Make sure the containers are running - the code is below in a block for you"
    Write-Output "docker ps -a"
    Write-Output "If they are read the logs - the code is below in a block for you"
    Write-Output "docker logs dbatools_SQL2019_1"
    Write-Output "docker logs dbatools_SQL2019-1_1"
}
# Run this first to make sure output width does not mess with output - Update output buffer size to prevent clipping in Visual Studio output window.
if( $Host -and $Host.UI -and $Host.UI.RawUI ) {
    $rawUI = $Host.UI.RawUI
    $oldSize = $rawUI.BufferSize
    $typeName = $oldSize.GetType( ).FullName
    $newSize = New-Object $typeName (500, $oldSize.Height)
    $rawUI.BufferSize = $newSize
  }

**NEVER EVER DO THIS IN PRODUCTION**
unless you need to delete all of your user databases for some reason


In [ ]:
Get-DbaDatabase -SqlInstance $SQL2 -ExcludeAllSystemDb | Remove-DbaDatabase -Confirm:$false

# Containers

dbatools is not able to create availability groups in containers with one command unfrotunately, so we will create an availability group below in a different way, which is cool because we can show you a bunch of other dbatools commands! If you are not using containers you can create an availability group with dbatools like this

```
$params = @{
    Primary = $SqlInstances[0]
    PrimarySqlCredential =  $SqlCredential 
    Secondary =   $SqlInstances[1]
    SecondarySqlCredential = $SqlCredential 
    Name = 'TheBeard_AG'
    Database = 'pubs','NorthWind','AdventureWorks2017'
    ClusterType = "None"
    SeedingMode = "Automatic"
    FailoverMode = "Manual"
    Confirm = $false
 }

# execute the command
 New-DbaAvailabilityGroup @params
```

There are many options that you can use here. Read the docs or get our book [beard.media\book](.\[beard.media\book)

Lets create an availability group to use with this notebook

First we shall create a master key on each instance

In [ ]:
New-DbaDbMasterKey -SqlInstance $SQL1 -Credential $SqlCredential -Confirm:$false
New-DbaDbMasterKey -SqlInstance $SQL2 -Credential $SqlCredential -Confirm:$false

# Certificate

Next we will create a certificate for the endpoints on SQL1, back it up and restore it on SQL2. You can do this for any certificate that you require by the way :-)

In [ ]:
New-DbaDbCertificate -SqlInstance $sql1 -Name hadr_cert -Subject hadr_cert -Confirm:$false

$EncryptionPassword = ConvertTo-SecureString -AsPlainText "Password4567!!" -force
Backup-DbaDbCertificate -SqlInstance $SQL1 -Suffix $null -Certificate hadr_cert -Path '/var/opt/mssql/backups/' -EncryptionPassword $EncryptionPassword -Confirm:$false


Now we will use `Get-DbaFile` to ge the certificate path and then restore the certificate to SQL2
This will prompt you to ask for input. If you did not wnat this, you can use `-Force` I am also showing verbose output here. All dbatools commands can output verbose information if you use the `-Verbose` switch

In [ ]:
$Certificate = (Get-DbaFile -SqlInstance $sql2 -Path '/var/opt/mssql/backups/' -FileType cer).FileName
Restore-DbaDbCertificate -SqlInstance $SQL2 -Path $Certificate -DecryptionPassword $EncryptionPassword  -Verbose -Confirm:$false


# Endpoints
Now that we have the certificates on each instance, we can create the endpoints using the certificate and start them

In [ ]:
New-DbaEndpoint -SqlInstance $sql1 -Name hadr_endpoint -Port 5022 -Certificate hadr_cert 
New-DbaEndpoint -SqlInstance $sql2 -Name hadr_endpoint -Port 5023 -Certificate hadr_cert

Start-DbaEndpoint -SqlInstance $SQL1 -EndPoint hadr_endpoint
Start-DbaEndpoint -SqlInstance $SQL2 -EndPoint hadr_endpoint

# Create the Availability Group

we have to use T-SQL to get the right information only because we are using containers

In [ ]:

$query = @"
CREATE AVAILABILITY GROUP [TheBeard_AG]
    WITH (CLUSTER_TYPE = NONE)
    FOR REPLICA ON
        N'beardsql01' WITH (
            ENDPOINT_URL = N'tcp://beardsql01:5022',
		    AVAILABILITY_MODE = ASYNCHRONOUS_COMMIT,
		    FAILOVER_MODE = MANUAL,
		    SEEDING_MODE = AUTOMATIC,
                    SECONDARY_ROLE (ALLOW_CONNECTIONS = ALL)
		    ),
        N'beardsql02' WITH ( 
		    ENDPOINT_URL = N'tcp://beardsql02:5023', 
		    AVAILABILITY_MODE = ASYNCHRONOUS_COMMIT,
		    FAILOVER_MODE = MANUAL,
		    SEEDING_MODE = AUTOMATIC,
		    SECONDARY_ROLE (ALLOW_CONNECTIONS = ALL)
		    );
		
ALTER AVAILABILITY GROUP [TheBeard_AG] GRANT CREATE ANY DATABASE;
"@

Invoke-DbaQuery -SqlInstance $sql1 -Query $query
Write-Output "Created AG"
$query = @"
ALTER AVAILABILITY GROUP [TheBeard_AG] JOIN WITH (CLUSTER_TYPE = NONE);
		 
ALTER AVAILABILITY GROUP [TheBeard_AG] GRANT CREATE ANY DATABASE;
"@

Invoke-DbaQuery -SqlInstance $sql2 -Query $query
Write-Output "Added SQL2 to the AG"

we can check that the availability groups on an instance using `Get-DbaAvailabilityGroup`

In [ ]:
Get-DbaAvailabilityGroup -SqlInstance $sql1 

Excellent, we have an availablity group - Lets check the replicas

In [ ]:
Get-DbaAgReplica -SqlInstance $sql1

We have two replicas - Right now they are not synchronising because there are no databases, You can use this command to see the rol up status for the replicas

Let's check the databases

In [ ]:
Get-DbaAgDatabase -SqlInstance $SQL1

There are no databases - Lets add the databases

## Compatability Level

In [ ]:
Get-DbaDbCompatibility -SqlInstance $Sql1,$sql2 | ft

# Recovery Mode

We need to set the recovery mode of the databases to full

In [ ]:
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeSystem | Set-DbaDbRecoveryModel -RecoveryModel Full -Confirm:$false

# Backup

Of course, having set the recovery model to full we need to back up the databases otherwise they are in PSUDOSIMPLE mode. You can see this. If we check the recovery model, it will show full

In [ ]:
Get-DbaDbRecoveryModel -SqlInstance $sql1 -Database AdventureWorks2017, Northwind, pubs | Format-Table

but if we use `Test-DbaDbRecoveryModel` you can see that the *Actual* recovery model is SIMPLE

In [ ]:
Test-DbaDbRecoveryModel -SqlInstance $sql1 -Database AdventureWorks2017, Northwind, pubs | Format-Table

If we backup the databases (like we did in the BackupsandRestores notebook

In [ ]:
Backup-DbaDatabase -SqlInstance $SQL1 -Path /var/opt/mssql/backups/SQL1 -Database  AdventureWorks2017, Northwind, pubs -Type Full -IgnoreFileChecks
Backup-DbaDatabase -SqlInstance $SQL1 -Path /var/opt/mssql/backups/SQL1 -Database  AdventureWorks2017, Northwind, pubs -Type Log -IgnoreFileChecks

and test our recovery model again

In [ ]:
Test-DbaDbRecoveryModel -SqlInstance $sql1 -Database AdventureWorks2017, Northwind, pubs | Format-Table

All our databases are now in FULL recovery :-)

Let's add them to the Availability Group

In [ ]:
$AddAgDbConfig = @{
    SqlInstance = $SqlInstances[0] 
    SqlCredential = $SqlCredential 
    AvailabilityGroup = 'TheBeard_AG' 
    Database = 'Northwind','Pubs','AdventureWorks2017' 
    Secondary = $SqlInstances[1] 
    SecondarySqlCredential = $SqlCredential 
    SeedingMode = 'Automatic' 
}
Add-DbaAgDatabase @AddAgDbConfig

Let's have a look at them


In [ ]:
$sql1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential
$sql2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
Get-DbaAgDatabase -SqlInstance $SQL1 | Format-Table

You can look in SSMS if you like :-)

![AG](.\images\ag.png )

## Suspending and resuming data movement

You can use dbatools to suspend and resume data movement for your Availability Group databases. The code below will prompt you to confirm that you wish to suspend them.

In [ ]:
Get-DbaAgDatabase -SqlInstance $sql1  | Suspend-DbaAgDbDataMovement

You can check that they are suspended using `Get-DbaAgDatabase` You will see that the `SynchronizationState` has changed to not synchronising and the `IsSuspended` property is set to true

In [ ]:
Get-DbaAgDatabase -SqlInstance $sql1 |Format-Table

If you want to see if any databases in your Availability Group are not synchronised you can use the code below (if you are looking on an asynchronous replica then you would change Synchronized to Synchronizing.

In [ ]:
Get-DbaAgDatabase -SqlInstance $SQL1 | Where SynchronizationState -ne 'Synchronized' | Format-Table

You can add the capability to resume data movement using dbatools as well with `Resume-DbaAgDbDataMovement` this time we will use `-Confirm:$false` so there are no prompts

In [ ]:
Get-DbaAgDatabase -SqlInstance $SQL1| Resume-DbaAgDbDataMovement -Confirm:$false

## Removing a database from an Availability Group

You can also remove a database from the availablity group. This code will prompt you to confirm the changing action which is usual dbatools behaviour


In [ ]:
Remove-DbaAgDatabase -SqlInstance $SQL1 -AvailabilityGroup TheBeard_AG -Database pubs

We can check the user databases on the instances with `Get-DbaDatabase`

In [ ]:
$sql1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential
$sql2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
Get-DbaDatabase -SqlInstance $SQL1, $SQL2 -ExcludeSystem | Select SqlInstance, Name,Status

You can see that the pubs database on the secondary is in restoring state.

You can also use `Get-DbaDatabase` to see the Availability Group synchronisation state and the Availability Group that it is joined to by getting some of the properties that are not displayed by default

In [ ]:
Get-DbaDatabase -SqlInstance $SQL1, $SQL2 -ExcludeSystem | Select SqlInstance, Name, AvailabilityDatabaseSynchronizationState, AvailabilityGroupName 

You can see that the pubs database is not joined to the Availability Group

# Failover

You can use dbatools to failover the Availability Group. Lets take a look at the Availability Group

In [ ]:
Get-DbaAvailabilityGroup -SqlInstance $SQL1

The primary replica for the Availability Group is SQL1. Lets fail it over to SQL2

In [ ]:
 Invoke-DbaAgFailover -SqlInstance $SQL2 -AvailabilityGroup TheBeard_AG

It has prompted us for confirmation and then told us that the cluster type that we have used for the containers does not support this type of failover so we need to use the force                     parameter

In [ ]:
 Invoke-DbaAgFailover -SqlInstance $SQL2 -AvailabilityGroup TheBeard_AG -Force

When we check the Availability Group on SQL1 again

In [ ]:
$sql1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential
$sql2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
$SqlInstances = $sql1,$sql2
Get-DbaAvailabilityGroup -SqlInstance $SQL1

You can see by comparing the output that the primary replica has swapped to the other container.

In [ ]:
Get-DbaDatabase -SqlInstance $SqlInstances -SqlCredential $SqlCredential -ExcludeSystem | Select SqlInstance, Name, AvailabilityDatabaseSynchronizationState, AvailabilityGroupName 

# Clean Up

The 99-CleanUpContainers notebook will remove the containers, files and directory - it will leave the image so you do not have to download it again!